In [1]:
# %% [Setup]
import os
import shutil
import subprocess
import cv2
import numpy as np
import torch
import lpips
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim

# Define paths
temp_folder = 'tmp_frames'
result_folder = 'results'

# Clean existing folders
for folder in [temp_folder, result_folder]:
    if os.path.isdir(folder):
        shutil.rmtree(folder)
    os.mkdir(folder)

print("✅ Directories set up successfully!")


c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Directories set up successfully!


In [ ]:
# %% [Select Video]
input_path = r'D:\University\IIT\Level 7\Final Year Project\MVP\EmotiLive\ESRGAN Model\inputs\ds3_1.mp4'

if not os.path.exists(input_path):
    raise ValueError(f"❌ Video file not found: {input_path}")

file_name = os.path.basename(input_path)
print(f"🎬 Selected Video: {file_name}")


🎬 Selected Video: ds3_1.mp4


In [ ]:
# %% [Extract Frames]
print("🔄 Extracting frames from video...")

cmd = [
    'ffmpeg',
    '-i', input_path,
    '-vf', 'fps=15', 
    '-q:v', '1',       
    f'{temp_folder}/frame_%08d.png'
]

process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
frame_count = len(os.listdir(temp_folder))

if process.returncode != 0 or frame_count == 0:
    raise RuntimeError("❌ Error extracting frames")

print(f"✅ Extracted {frame_count} frames successfully!")


🔄 Extracting frames from video...
✅ Extracted 196 frames successfully!


In [4]:
# %% [Enhance Frames with Real-ESRGAN+]
print("🚀 Enhancing frames using Real-ESRGAN+...")

cmd = [
    'python', 'inference_realesrgan.py',
    '-n', 'RealESRGAN_x4plus_anime_6B',  
    '-i', temp_folder,
    '--outscale', '4',
    '--face_enhance' 
]

process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

if process.returncode != 0:
    raise RuntimeError("❌ Error enhancing frames")

print("✅ Frame enhancement complete!")


🚀 Enhancing frames using Real-ESRGAN+...
✅ Frame enhancement complete!


In [5]:
# %% [Denoise Enhanced Frames]
print("🧹 Applying denoising on enhanced frames...")

for frame_file in os.listdir(temp_folder):
    if frame_file.endswith(".png"):
        img_path = os.path.join(temp_folder, frame_file)
        img = cv2.imread(img_path)

        # Apply Non-Local Means Denoising
        denoised_img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

        # Save back the denoised frame
        cv2.imwrite(img_path, denoised_img)

print("✅ Denoising complete!")


🧹 Applying denoising on enhanced frames...
✅ Denoising complete!


In [ ]:
# %% [Recreate Video with Higher Bitrate]
print("🎥 Recreating video with high bitrate...")

output_video = os.path.join(result_folder, f"enhanced_{file_name}")
fps = 15 

cmd = [
    'ffmpeg',
    '-framerate', str(fps),
    '-i', os.path.join(temp_folder, 'frame_%08d.png'),
    '-c:v', 'libx264',
    '-preset', 'slow',
    '-crf', '18',  
    '-b:v', '8000k',  
    '-r', str(fps),
    '-pix_fmt', 'yuv420p',
    output_video
]

process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

print("FFmpeg Output:", process.stdout)
print("FFmpeg Error:", process.stderr)

if process.returncode != 0:
    raise RuntimeError(f"❌ Error recreating video. FFmpeg Error:\n{process.stderr}")

print(f"✅ Enhanced video saved as: {output_video}")


🎥 Recreating video with high bitrate...
FFmpeg Output: 
FFmpeg Error: ffmpeg version N-118892-ge5d62e20c8-20250321 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (crosstool-NG 1.27.0.18_7458341)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-libxml2 --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --en

In [ ]:
# %% [Define Quality Metrics]
print("📊 Initializing quality metrics...")

lpips_loss = lpips.LPIPS(net='alex')

def compute_psnr(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    return 100 if mse == 0 else 20 * np.log10(255.0 / np.sqrt(mse))

def compute_ssim(image1, image2):
    return ssim(image1, image2, data_range=255, channel_axis=-1)

def compute_mse(image1, image2):
    return np.mean((image1 - image2) ** 2)

def compute_lpips(image1, image2):
    image1 = torch.from_numpy(image1).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    image2 = torch.from_numpy(image2).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    return lpips_loss(image1, image2).item()

print("✅ Quality metrics initialized!")


📊 Initializing quality metrics...
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\lpips\weights\v0.1\alex.pth
✅ Quality metrics initialized!


In [10]:
# %% [Initialize VGG Perceptual Loss]
vgg_model = models.vgg19(pretrained=True).features[:16].eval().to("cuda" if torch.cuda.is_available() else "cpu")

def compute_vgg_loss(image1, image2):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image1 = transform(image1).unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
    image2 = transform(image2).unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
    
    with torch.no_grad():
        features1 = vgg_model(image1)
        features2 = vgg_model(image2)
    
    return nn.functional.mse_loss(features1, features2).item()


In [13]:
# %% [Import Necessary Libraries]
import torch
import lpips
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim

# Initialize LPIPS loss function
lpips_loss = lpips.LPIPS(net='alex')

# %% [Define Quality Metrics Functions]
def compute_psnr(image1, image2):
    """ Compute Peak Signal-to-Noise Ratio (PSNR) """
    mse = np.mean((image1 - image2) ** 2)
    return 100 if mse == 0 else 20 * np.log10(255.0 / np.sqrt(mse))

def compute_ssim(image1, image2):
    """ Compute Structural Similarity Index (SSIM) """
    return ssim(image1, image2, data_range=255, channel_axis=-1)

def compute_mse(image1, image2):
    """ Compute Mean Squared Error (MSE) """
    return np.mean((image1 - image2) ** 2)

def compute_lpips(image1, image2):
    """ Compute Perceptual Loss using LPIPS """
    image1 = torch.from_numpy(image1).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    image2 = torch.from_numpy(image2).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    return lpips_loss(image1, image2).item()

def compute_vgg_loss(image1, image2):
    """ Dummy VGG Loss function (Future implementation) """
    return compute_lpips(image1, image2)  # Using LPIPS as a proxy

print("✅ Quality metric functions initialized!")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\lpips\weights\v0.1\alex.pth
✅ Quality metric functions initialized!


In [15]:
# %% [Compare Videos]
def compare_videos(original_video_path, enhanced_video_path):
    original_video = cv2.VideoCapture(original_video_path)
    enhanced_video = cv2.VideoCapture(enhanced_video_path)

    psnr_values, ssim_values, mse_values, lpips_values, vgg_loss_values = [], [], [], [], []
    frame_count = 0

    while original_video.isOpened() and enhanced_video.isOpened():
        ret1, frame1 = original_video.read()
        ret2, frame2 = enhanced_video.read()

        if not ret1 or not ret2:
            break  # Stop when no more frames

        # Ensure both frames have the same dimensions
        frame1 = cv2.resize(frame1, (640, 640))
        frame2 = cv2.resize(frame2, (640, 640))

        frame_count += 1
        print(f"📸 Processing frame {frame_count}...")

        # Compute quality metrics
        psnr = compute_psnr(frame1, frame2)
        ssim_value = compute_ssim(frame1, frame2)
        mse_value = compute_mse(frame1, frame2)
        lpips_value = compute_lpips(frame1, frame2)
        vgg_loss_value = compute_vgg_loss(frame1, frame2)

        # Append results
        psnr_values.append(psnr)
        ssim_values.append(ssim_value)
        mse_values.append(mse_value)
        lpips_values.append(lpips_value)
        vgg_loss_values.append(vgg_loss_value)

    original_video.release()
    enhanced_video.release()

    # Print Average Results
    print("\n=== 🎯 Video Quality Comparison Results ===")
    print(f"📌 Average PSNR:  {np.mean(psnr_values):.2f} dB (Higher is better)")
    print(f"📌 Average SSIM:  {np.mean(ssim_values):.4f} (1.0 = Perfect match)")
    print(f"📌 Average MSE:   {np.mean(mse_values):.2f} (Lower is better)")
    print(f"📌 Average LPIPS: {np.mean(lpips_values):.4f} (Lower is better, deep-learning based)")
    print(f"📌 Average VGG Loss: {np.mean(vgg_loss_values):.4f} (Lower means better perceptual similarity)")

# Run comparison
compare_videos(input_path, output_video)


📸 Processing frame 1...
📸 Processing frame 2...
📸 Processing frame 3...
📸 Processing frame 4...
📸 Processing frame 5...
📸 Processing frame 6...
📸 Processing frame 7...
📸 Processing frame 8...
📸 Processing frame 9...
📸 Processing frame 10...
📸 Processing frame 11...
📸 Processing frame 12...
📸 Processing frame 13...
📸 Processing frame 14...
📸 Processing frame 15...
📸 Processing frame 16...
📸 Processing frame 17...
📸 Processing frame 18...
📸 Processing frame 19...
📸 Processing frame 20...
📸 Processing frame 21...
📸 Processing frame 22...
📸 Processing frame 23...
📸 Processing frame 24...
📸 Processing frame 25...
📸 Processing frame 26...
📸 Processing frame 27...
📸 Processing frame 28...
📸 Processing frame 29...
📸 Processing frame 30...
📸 Processing frame 31...
📸 Processing frame 32...
📸 Processing frame 33...
📸 Processing frame 34...
📸 Processing frame 35...
📸 Processing frame 36...
📸 Processing frame 37...
📸 Processing frame 38...
📸 Processing frame 39...
📸 Processing frame 40...
📸 Process